In [1]:
from datetime import datetime
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
# Đọc dữ liệu từ tệp
dateparse = lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M')
df = pd.read_csv("C:\\Users\\7\\PycharmProjects\\baitaplon\OnlineRetail.csv", parse_dates=['InvoiceDate'], date_parser=dateparse,
                 encoding='unicode_escape')
print(df.head())

C:\Users\7\AppData\Local\Temp\ipykernel_9004\499704797.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv("C:\\Users\\7\\PycharmProjects\\baitaplon\OnlineRetail.csv", parse_dates=['InvoiceDate'], date_parser=dateparse,


  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [17]:

df1 = df.dropna(subset=['CustomerID'])
print("Sau khi loại bỏ các dòng có giá trị 'CustomerID' bị thiếu:")
print(df1.head())  # Hiển thị 5 dòng đầu tiên của DataFrame sau khi loại bỏ
#Tạo ma trận khách hàng-sản phẩm từ DataFrame đã được tiền xử lý.
#  Ma trận này có các hàng là các "CustomerID", các cột là các "StockCode",
#  và các giá trị là tổng số lượng sản phẩm mà khách hàng đó đã mua:
customer_item_matrix = df1.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', aggfunc='sum')
print("Ma trận khách hàng-sản phẩm:")
print(customer_item_matrix.head())  # Hiển thị ma trận khách hàng-sản phẩm
#Xử lý ma trận khách hàng-sản phẩm thành ma trận nhị phân, 
# trong đó 1 biểu thị khách hàng đã mua sản phẩm đó và 0 biểu thị khách hàng không mua sản phẩm đó:
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
print("Ma trận khách hàng-sản phẩm đã được xử lý:")
print(customer_item_matrix.head())  # Hiển thị ma trận khách hàng-sản phẩm đã được xử lý

Sau khi loại bỏ các dòng có giá trị 'CustomerID' bị thiếu:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
Ma trận khách hàng-sản phẩm:
StockCode   10002  10080  10120  10123C  10124A  10124G  10125  10133  10135  \
CustomerID           

In [18]:
# Tính độ tương đồng giữa người dùng
#Sử dụng hàm cosine_similarity từ thư viện sklearn để tính độ tương đồng cosine giữa các hàng của ma trận khách hàng-sản phẩm:
user_to_user_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix))
#Đặt tên các cột của DataFrame user_to_user_sim_matrix bằng các giá trị trong cột index của ma trận khách hàng-sản phẩm:
user_to_user_sim_matrix.columns = customer_item_matrix.index
#Thêm cột "CustomerID" vào DataFrame user_to_user_sim_matrix:
user_to_user_sim_matrix['CustomerID'] = customer_item_matrix.index
#Đặt cột "CustomerID" làm index của DataFrame user_to_user_sim_matrix:
user_to_user_sim_matrix = user_to_user_sim_matrix.set_index('CustomerID')

print("Ma trận độ tương đồng giữa các người dùng:")
print(user_to_user_sim_matrix.head())  # Hiển thị ma trận độ tương đồng giữa các người dùng

Ma trận độ tương đồng giữa các người dùng:
CustomerID  12346.0   12347.0   12348.0   12349.0   12350.0   12352.0  \
CustomerID                                                              
12346.0         0.0  0.000000  0.000000  0.000000  0.000000  0.000000   
12347.0         0.0  1.000000  0.063022  0.046130  0.047795  0.038814   
12348.0         0.0  0.063022  1.000000  0.024953  0.051709  0.027995   
12349.0         0.0  0.046130  0.024953  1.000000  0.056773  0.138314   
12350.0         0.0  0.047795  0.051709  0.056773  1.000000  0.031846   

CustomerID  12353.0   12354.0   12355.0   12356.0  ...  18273.0  18274.0  \
CustomerID                                         ...                     
12346.0         0.0  0.000000  0.000000  0.000000  ...      0.0      0.0   
12347.0         0.0  0.025876  0.136641  0.094742  ...      0.0      0.0   
12348.0         0.0  0.027995  0.118262  0.146427  ...      0.0      0.0   
12349.0         0.0  0.030737  0.032461  0.144692  ...      0.0  

In [19]:
# Tính độ tương đồng giữa sản phẩm
#Sử dụng hàm cosine_similarity từ thư viện sklearn để tính độ tương đồng cosine 
#giữa các cột của ma trận khách hàng-sản phẩm (sau khi đã chuyển vị):
item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix.T))
#Đặt tên các cột của DataFrame item_item_sim_matrix bằng các giá trị trong cột
# index của ma trận khách hàng-sản phẩm (sau khi đã chuyển vị):
item_item_sim_matrix.columns = customer_item_matrix.T.index
#Thêm cột "StockCode" vào DataFrame item_item_sim_matrix:
item_item_sim_matrix['StockCode'] = customer_item_matrix.T.index
#Đặt cột "StockCode" làm index của DataFrame item_item_sim_matrix:
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')
# in kết quả của ma trận độ tương đồng giữa các người dùng và ma trận độ tương đồng giữa các sản phẩm bằng cách sử dụng hàm print():
print("Ma trận độ tương đồng giữa các sản phẩm:")
print(item_item_sim_matrix.head())  # Hiển thị ma trận độ tương đồng giữa các sản phẩm

Ma trận độ tương đồng giữa các sản phẩm:
StockCode     10002  10080     10120    10123C  10124A    10124G     10125  \
StockCode                                                                    
10002      1.000000    0.0  0.094868  0.091287     0.0  0.000000  0.090351   
10080      0.000000    1.0  0.000000  0.000000     0.0  0.000000  0.032774   
10120      0.094868    0.0  1.000000  0.115470     0.0  0.000000  0.057143   
10123C     0.091287    0.0  0.115470  1.000000     0.0  0.000000  0.164957   
10124A     0.000000    0.0  0.000000  0.000000     1.0  0.447214  0.063888   

StockCode     10133     10135     11001  ...  90214Y  90214Z  BANK CHARGES  \
StockCode                                ...                                 
10002      0.063246  0.098907  0.095346  ...     0.0     0.0           0.0   
10080      0.045883  0.047836  0.000000  ...     0.0     0.0           0.0   
10120      0.060000  0.041703  0.060302  ...     0.0     0.0           0.0   
10123C     0.000000  0

In [20]:

# Hàm gợi ý sản phẩm dựa trên mã sản phẩm
def recommend_items_by_stockcode(stockcode, num_items=10):
    if stockcode not in item_item_sim_matrix.index:
        return None

    top_similar_items = list(
        item_item_sim_matrix
        .loc[stockcode]
        .sort_values(ascending=False)
        .iloc[:num_items]
        .index
    )

    recommended_items = df.loc[
        df['StockCode'].isin(top_similar_items),
        ['StockCode', 'Description']
    ].drop_duplicates().set_index('StockCode')

    return recommended_items

In [21]:
# Hàm gợi ý sản phẩm dựa trên từ khóa
def recommend_items_by_keyword(keyword, num_items=10):
    df_filtered = df.dropna(subset=['Description'])
    matching_items = df_filtered[df_filtered['Description'].str.contains(keyword, case=False)]['StockCode'].unique()

    if len(matching_items) == 0:
        return None

    top_similar_items = list(
        item_item_sim_matrix
        .loc[matching_items]
        .mean()
        .sort_values(ascending=False)
        .iloc[:num_items]
        .index
    )

    recommended_items = df.loc[
        df['StockCode'].isin(top_similar_items),
        ['StockCode', 'Description']
    ].drop_duplicates().set_index('StockCode')

    return recommended_items

In [22]:

# Quá trình gợi ý sản phẩm
while True:
    user_input = input("Nhập mã sản phẩm hoặc từ khóa (hoặc 'q' để thoát): ")
    if user_input == 'q':
        break

    num_items = int(input("Nhập số sản phẩm gợi ý: "))

    if user_input.isdigit():
        recommended_items = recommend_items_by_stockcode(user_input, num_items)
    else:
        recommended_items = recommend_items_by_keyword(user_input, num_items)

    if recommended_items is None:
        print("Không tìm thấy gợi ý sản phẩm cho mã hoặc từ khóa '{}'.".format(user_input))
    else:
        print("Các sản phẩm tương tự:")
        print(recommended_items)

Các sản phẩm tương tự:
                                  Description
StockCode                                    
84854                     GIRLY PINK TOOL SET
84360        TABLE LAMP WHITE SHADE WOOD BASE
40001             WHITE BAMBOO RIBS LAMPSHADE
84358              FLOOR LAMP SHADE WOOD BASE
40003             WHITE BAMBOO RIBS LAMPSHADE
85161                  ACRYLIC GEOMETRIC LAMP
47351B                  BLUE  VOILE LAMPSHADE
85161                         samples/damages
44092C        PURPLE/COPPER HANGING LAMPSHADE
44092B     BLUE WHITE PLASTIC RINGS LAMPSHADE
84854                                     NaN
23104              IVORY PANTRY HANGING LAMP 
23104                IVORY CAFE HANGING LAMP 
40003                                     NaN
40001                                   check
